### 🛡️ Lecture: Understanding and Preventing SQL Injection in Python with pyodbc
**Learning Objectives**

By the end of this lecture, students will:

    Understand what SQL Injection is and why it’s dangerous

    See real examples of injection attacks

    Learn how to identify vulnerable code patterns

    Apply parameterized queries using pyodbc to prevent attacks

---

### 🔍 What is SQL Injection?

**Definition**

SQL Injection is a type of security vulnerability that allows an attacker to interfere with the queries your application sends to a database.

🧨 What can an attacker do?

    Bypass login screens

    Read, modify, or delete data

    Drop entire tables or databases

    Execute arbitrary database commands

🦠 Why does it happen?

Because the application concatenates user input directly into SQL strings.

---

### 🧪 Setting Up a Demo Table

We’ll use a simple Users table to simulate login logic.
💬 What we'll do:

    Create a Users table with fake usernames and passwords

    Simulate login functionality (both unsafe and safe)

    Demonstrate how an attacker can exploit poor code

In [ ]:
import pyodbc

server_name = 'LAPTOP-OM16N5V6'  # replace with your server name
database_name = 'AI24'           # replace with your database name

# Define connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=localhost;'
    f'SERVER={server_name};'
    f'DATABASE={database_name};'
    'Trusted_Connection=yes;'
)

print("Connected!")

We will keep building on the Library schema defined in the previous notebook.

In [ ]:
cursor = conn.cursor()

# Drop table if it exists

cursor.execute("IF OBJECT_ID('Library.Users', 'U') IS NOT NULL DROP TABLE Library.Users;")

# Create TABLE Users

cursor.execute('''
                CREATE TABLE Library.Users (
                    UserID INT PRIMARY KEY IDENTITY,
                    Username NVARCHAR(100),
                    Password NVARCHAR(100)
                );
              ''')

cursor.executemany(
    "INSERT INTO Library.Users (Username, Password) VALUES (?, ?);",
    [
        ('alice', 'password123'),
        ('bob', 'qwerty'),
        ('charlie', 'letmein'),
    #    ('alice', "' OR '1'='1")        -- uncomment to see how this affects the safe login demo
    ]
)

conn.commit()

### ❌ Unsafe Login Function (Vulnerable to SQL Injection)
💬 Why is this dangerous?

The username and password values are directly inserted into the SQL string. If the user includes malicious SQL in their input, it will be executed by the server.

In [ ]:
def unsafe_login_method(username, password):
    
    query = f"SELECT * FROM Library.Users WHERE Username = '{username}' AND Password = '{password}';"
    print("[!] Executing SQL:", query)
    cursor.execute(query)
    return cursor.fetchall()

In [ ]:
# legitimate login
# You should get back one row for Alice.

unsafe_login_method('alice', 'password123')

In [ ]:
# SQL Injection Attack: Bypass Login
# This may return all users, showing how the login was bypassed.

unsafe_login_method('alice', "' OR '1'='1")

### 🔥 More Dangerous: DELETE via Injection

If your app executes any other kind of SQL besides SELECT — say, DELETE — this becomes catastrophic.

In [ ]:
# NEVER define a function like this

def delete_user(username):
    query = f"DELETE FROM Library.Users WHERE Username LIKE '{username}'"
    print("[!] Executing SQL:", query)
    cursor.execute(query)
    conn.commit()

In [ ]:
# This will first delete the user Alice, but then also the entire table!

delete_user("alice'; DROP TABLE Library.Users;--")

Question: why does this end with "--"?

**Before proceeding,  re-run the code that creates, and insert data into, our table Library.Users**

---

### ✅ Safe Login Using Parameterized Queries

Using ? placeholders ensures user input is treated as data, not as SQL code.

In [ ]:
def safe_login(username, password):
    query = "SELECT * FROM Library.Users WHERE Username = ? AND Password = ?"
    print(f"[+] Executing safe parameterized query: {query}")
    cursor.execute(query, (username, password))
    return cursor.fetchall()

🧪 Test the Same Attack Again (Safely)

In [ ]:
safe_login('alice', "' OR '1'='1")

✅ This will return nothing — the injection is now treated as a literal string, not part of the SQL logic.

---


### Final Thoughts

        SQL injection is one of the oldest and most common attack vectors.

        It is easy to exploit but also easy to prevent with the right habits.

        If your application accepts user input, assume it’s hostile — and sanitize or parameterize accordingly.

### Homework / Practice Ideas

        Modify the unsafe login function to try different injections.

        Add a new table and try injecting destructive SQL (e.g. DROP, UPDATE).

        Use executemany() to safely insert multiple users.